In [ ]:
# %cd f-threshold-select

In [ ]:
# !git clone https://github.com/fabian-sp/GGLasso.git

In [ ]:
import gglasso
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import os
import matplotlib.pyplot as plt

from numpy.linalg import matrix_rank
from matplotlib.pyplot import figure
from scipy import stats
from scipy.linalg import eigh
from numpy import genfromtxt
from datetime import datetime

from gglasso.solver.admm_solver import ADMM_MGL
from gglasso.problem import glasso_problem

from gglasso.helper.data_generation import generate_precision_matrix, group_power_network, sample_covariance_matrix
from gglasso.helper.basic_linalg import adjacency_matrix
from gglasso.helper.data_generation import time_varying_power_network, sample_covariance_matrix
from gglasso.helper.experiment_helper import lambda_grid, discovery_rate, error
from gglasso.helper.utils import get_K_identity
from gglasso.helper.experiment_helper import plot_evolution, plot_deviation, surface_plot, single_heatmap_animation
from gglasso.helper.model_selection import aic, ebic, K_single_grid

### Read data

In [ ]:
corr = []

for i in range(0, 950):
    corr.append(genfromtxt("/storage/groups/bds01/datasets/brains/corr_matrices/corr{0}.csv".format(i), delimiter=','))

corr = np.array(corr)
corr.shape

### Single GL

In [ ]:
lambda1_range = np.logspace(0, -3, 2)

In [ ]:
N = corr.shape[1]

In [ ]:
est_uniform, est_indv, statistics = K_single_grid(corr[:1, :], lambda1_range, N, 
                                                  method = 'eBIC', gamma = 0.3, 
                                                  latent = False, use_block = True)

In [ ]:
K = est_uniform["Theta"].shape[0]

In [ ]:
os.mkdir("/storage/groups/bds01/datasets/brains/est_uniform{0}/".format(K))
os.mkdir("/storage/groups/bds01/datasets/brains/est_individ{0}/".format(K))

In [ ]:
# dump matrices into csv
for i in range(0, K):
    np.savetxt("/storage/groups/bds01/datasets/brains/est_uniform{0}/est_uniform{1}.csv".format(K, i), est_uniform["Theta"][i], 
               delimiter=",", header='')
    np.savetxt("/storage/groups/bds01/datasets/brains/est_uniform{0}/est_individ{1}.csv".format(K, i), est_indv["Theta"][i], 
               delimiter=",", header='')

In [ ]:
# !jupyter nbconvert --to script --no-prompt SGL.ipynb

In [ ]:
with open("/storage/groups/bds01/datasets/brains/est_uniform50/statistics{0}.txt".format(K), 'w') as f:
    print(statistics, file=f)

In [ ]:
os.mkdir("/storage/groups/bds01/datasets/brains/est_uniform{0}/".format(K))

### Read solution

In [ ]:
Theta = []

for i in range(0, 50):
    Theta.append(genfromtxt("/storage/groups/bds01/datasets/brains/est_uniform50/est_uniform{0}.csv".format(i), delimiter=','))

Theta = np.array(Theta)
Theta.shape

In [ ]:
K = Theta.shape[0]

### Plot the solution

In [ ]:
# for i in range(0, K):
#     fig, axes = plt.subplots(1, 2, sharex=True, figsize=(30,30))

#     ax = sns.heatmap(sub_corr[i, :], ax=axes[0], center=0, vmin = -0.5, vmax = 0.5, 
#                      square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
#     ax = sns.heatmap(Theta[i, :], ax=axes[1], center=0, vmin = -0.5, vmax = 0.5,
#                      square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
#     fig.savefig("plots/SGL_plots/SGL_heatmap{0}.png".format(i))

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
sex = pd.read_csv("/storage/groups/bds01/datasets/brains/sex.csv")
age = pd.read_csv("/storage/groups/bds01/datasets/brains/age.csv")

In [ ]:
sex_50 = np.array(sex.iloc[:50,])
age_50 = np.array(age.iloc[:50,])

### Predict sex

In [ ]:
X, y = Theta, sex_50

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print('X_train: ',X_train.shape)
print('X_test: ',X_test.shape)
print('Y_train: ',Y_train.shape)
print('Y_test: ',Y_test.shape)

In [ ]:
# Converting the 3D matrix to 2D matrix
print('X_train: ',X_train.shape)
print('X_test: ',X_test.shape)

number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]

X_train_flatten = X_train.reshape(number_of_train, X_train.shape[1]*X_train.shape[2])
X_test_flatten = X_test .reshape(number_of_test, X_test.shape[1]*X_test.shape[2])

print("X train flatten", X_train_flatten.shape)
print("X test flatten", X_test_flatten.shape)

x_train = X_train_flatten
x_test = X_test_flatten
y_train = Y_train
y_test = Y_test

print("x train: ", x_train.shape)
print("x test: ", x_test.shape)
print("y train: ", y_train.shape)
print("y test: ", y_test.shape)

In [ ]:
# create model and train test accuracy print
logreg = LogisticRegression(random_state = 42, max_iter= 150, penalty='l2')
logreg.fit(x_train, y_train.ravel())
print("Test accuracy: {} ".format(logreg.score(x_test, y_test)))